In [1]:
import pipeline
import pandas as pd
import importlib
importlib.reload(pipeline)

<module 'pipeline' from '/Users/piyushtank/Dropbox/mscapp/spring_20/machine_learning/project/covid/git/ml-covid/scripts/pipeline.py'>

In [2]:
filepath = '../data/covid_df_protocol_3.pkl'
df = pd.read_pickle(filepath)

In [3]:
weeks_for_cv = [0,1,2,3,4,5,6]
n_weeks_prediction = 2
results = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    pipeline.sanity_check(df_train, df_test)
    #df_train = df_train.dropna()
    #df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Daily New Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Daily New Cases')
    dct = pipeline.train_and_evaluate(X_train, y_train, X_test, y_test)
    results['Until week: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-29
Success: Features match
Success: No NAs remain


/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11450362200.127068, tolerance: 4971216.729031145
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
48                         Country_China -5361.164445
5                        StringencyIndex -5154.569630
0                              Intercept  2169.785394
203  H1_Public information campaigns_2.0  1807.948329
198                E1_Income support_2.0  1675.665983
181        C5_Close public transport_0.0 -1477.643076
199          E2_Debt/contract relief_0.0 -1281.721318
44                        Country_Canada -1176.620134
207                H2_Testing policy_3.0  1145.013971
172             C2_Workplace closing_3.0  1022.256151

Bias: 1227636.24
Mean squared error: 3550204.71
RSS: 5275604205.47
Variance score: 0.51

R2 score: 0.53

Ridge: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
0                              Intercept  38

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8864240634.811012, tolerance: 4407638.045617789
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
5                        StringencyIndex -4896.078564
48                         Country_China -4391.837488
0                              Intercept  2623.014870
200                E1_Income support_2.0  1863.681673
205  H1_Public information campaigns_2.0  1713.680376
183        C5_Close public transport_0.0 -1427.082308
201          E2_Debt/contract relief_0.0 -1371.889861
209                H2_Testing policy_3.0  1234.736490
44                        Country_Canada -1074.821847
141                  Country_South Korea -1038.946660

Bias: 1179719.40
Mean squared error: 2655660.63
RSS: 5226340120.05
Variance score: 0.50

R2 score: 0.55

Ridge: Features with highest magnitude                                coefficients in absolute value
                                 Feature  Coefficient
0                              Intercept  36

...

Lasso: Features with highest magnitude                                coefficients in absolute value
                           Feature  Coefficient
0                        Intercept  3669.279825
5                  StringencyIndex -3022.184075
199          E1_Income support_2.0  2284.304630
48                   Country_China -1823.774822
200    E2_Debt/contract relief_0.0 -1529.407348
209          H2_Testing policy_3.0  1373.480549
79                   Country_India  1299.964411
140            Country_South Korea -1141.856538
182  C5_Close public transport_0.0  -935.142557
8             EconomicSupportIndex  -920.342344

Bias: 877737.93
Mean squared error: 3415913.81
RSS: 6913809552.48
Variance score: 0.43

R2 score: 0.49

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  5871.967427
199        E1_Income support_2.0  2191.870561
5                Stringency

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3827016746.5709305, tolerance: 1649416.3287543827
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  3601.709064
5                StringencyIndex -2296.998423
200        E1_Income support_2.0  2203.274237
210        H2_Testing policy_3.0  1508.451370
201  E2_Debt/contract relief_0.0 -1485.260396
80                 Country_India  1203.049110
141          Country_South Korea -1153.747476
49                 Country_China  -997.989592
198        E1_Income support_0.0  -888.552680
8           EconomicSupportIndex  -823.314635

Bias: 670615.67
Mean squared error: 4083572.75
RSS: 8318237686.73
Variance score: 0.40

R2 score: 0.42

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  6374.209750
200        E1_Income support_2.0  2247.553568
198        E1_Income support_0.0 -1790.745451
201

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2040590255.115909, tolerance: 840830.0780238091
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  2490.551916
201        E1_Income support_2.0  1731.079290
211        H2_Testing policy_3.0  1246.042171
202  E2_Debt/contract relief_0.0 -1158.794707
5                StringencyIndex -1117.720113
142          Country_South Korea  -923.350637
87                 Country_Italy   704.028891
143                Country_Spain   701.976385
199        E1_Income support_0.0  -648.402821
81                 Country_India   647.730252

Bias: 383009.73
Mean squared error: 5066488.06
RSS: 10381234030.04
Variance score: 0.38

R2 score: 0.32

Ridge: Features with highest magnitude                                coefficients in absolute value
                         Feature  Coefficient
0                      Intercept  6005.365640
201        E1_Income support_2.0  2039.226468
199        E1_Income support_0.0 -1653.691965
20

In [4]:
metrics_r2 = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_r2.loc[i,key] = results[mkey][key][4]
metrics_r2

,Lasso,Ridge,Linear Regression
0,0.529045,0.538256,0.541861
1,0.554805,0.56438,0.567738
2,0.544722,0.557039,0.562171
3,0.520936,0.536062,0.540169
4,0.485592,0.499308,0.502112
5,0.419886,0.434213,0.435184
6,0.319501,0.343298,0.343906


In [5]:
metrics_mse = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_mse.loc[i,key] = results[mkey][key][1]
metrics_mse

,Lasso,Ridge,Linear Regression
0,3.5502e+06,3.48077e+06,3.4536e+06
1,2.65566e+06,2.59855e+06,2.57851e+06
2,2.61214e+06,2.54147e+06,2.51203e+06
3,2.99953e+06,2.90483e+06,2.87911e+06
4,3.41591e+06,3.32483e+06,3.30621e+06
5,4.08357e+06,3.98273e+06,3.97588e+06
6,5.06649e+06,4.88932e+06,4.88478e+06


## with Grid Search for Lasso and Ridge

In [6]:
weeks_for_cv = [0,1,2,3,4,5,6]
n_weeks_prediction = 2
results_grid = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    pipeline.sanity_check(df_train, df_test)
    #df_train = df_train.dropna()
    #df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')
    dct = pipeline.train_and_evaluate_w_grid(X_train, y_train, X_test, y_test)
    results_grid['Until week: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-29
Success: Features match
Success: No NAs remain


/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12726093503447.812, tolerance: 4685249832.446781
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12862069322488.62, tolerance: 4604265618.223251
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12599971566052.13, tolerance: 4599490281.589481
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not con

{'mean_fit_time': array([1.32921343, 1.37220669, 1.41015511, 1.37536731, 1.44732537]), 'std_fit_time': array([0.03655882, 0.03136083, 0.0245606 , 0.06031361, 0.17400478]), 'mean_score_time': array([0.00444417, 0.004565  , 0.00459213, 0.00482273, 0.00422068]), 'std_score_time': array([1.27246187e-03, 4.64245410e-04, 4.58928489e-04, 8.20658222e-04,
       9.95278523e-05]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-2.54575013, -2.48858474, -2.46806995, -2.46483614, -2.45195115]), 'split1_test_score': array([-0.08001423, -0.09263021, -0.10402982, -0.09967808, -0.09697123]), 'split2_test_score': array([-37.88553826, -28.9684314 , -21.328622  , -14.93023786,
        -9.78823959]), 'split3_test_score': array([-2.12900363, -1.97978972, -1.89111314, -1.83

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8849520047414.63, tolerance: 3170478422.125867
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8912436787547.941, tolerance: 3094368474.6145883
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8706965532890.748, tolerance: 3096297560.8648667
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not con

{'mean_fit_time': array([1.50294085, 1.33062611, 1.37200875, 1.33055916, 1.30987406]), 'std_fit_time': array([0.0263423 , 0.05387388, 0.03344795, 0.02643315, 0.01007303]), 'mean_score_time': array([0.00514812, 0.0043118 , 0.00447636, 0.00433798, 0.00433249]), 'std_score_time': array([1.30668249e-03, 9.82723092e-05, 3.35690968e-04, 1.25408300e-04,
       2.31993377e-04]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-2.04642698, -2.03845149, -2.02909953, -2.02060536, -2.02127656]), 'split1_test_score': array([0.0351389 , 0.02650834, 0.0219254 , 0.02306484, 0.02407996]), 'split2_test_score': array([-17.77646608, -11.26432211,  -6.32712803,  -2.95856659,
        -1.16530183]), 'split3_test_score': array([-3.26653053, -3.22308771, -3.20040668, -3.2026344

...

{'mean_fit_time': array([1.22258077, 1.20092516, 1.35597568, 1.29055796, 1.19145436]), 'std_fit_time': array([0.07556357, 0.06362871, 0.13620711, 0.09683656, 0.01308505]), 'mean_score_time': array([0.00418344, 0.00460024, 0.00485139, 0.0041616 , 0.00420775]), 'std_score_time': array([0.00041855, 0.00067602, 0.00045206, 0.00011376, 0.00010369]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([-0.2888484 , -0.28317159, -0.27925966, -0.27041448, -0.25966014]), 'split1_test_score': array([0.13235399, 0.13629754, 0.13452865, 0.13465497, 0.13504402]), 'split2_test_score': array([-952.38740232, -737.68917669, -550.29606037, -390.3024759 ,
       -257.85427019]), 'split3_test_score': array([-15.97261646, -15.89828448, -15.81704907, -15.72822818,
       -15.6092

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 679264028565.0579, tolerance: 220577347.76082414
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 695805555545.3573, tolerance: 238725899.83855957
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 646483968702.2114, tolerance: 229526311.60403845
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not c

{'mean_fit_time': array([1.18461924, 1.17749934, 1.20936494, 1.23378601, 1.14121141]), 'std_fit_time': array([0.05530088, 0.04180861, 0.11843762, 0.15318098, 0.04212641]), 'mean_score_time': array([0.0041554 , 0.00400753, 0.00414386, 0.00401258, 0.00433431]), 'std_score_time': array([1.60170162e-04, 5.43935986e-05, 9.25306898e-05, 4.50773815e-05,
       3.12525731e-04]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([0.20191663, 0.21420096, 0.22020369, 0.2249503 , 0.22948802]), 'split1_test_score': array([0.04082807, 0.03701767, 0.03554229, 0.03781321, 0.03786972]), 'split2_test_score': array([-5231.42803452, -4565.56973341, -3946.18396316, -3371.10312418,
       -2841.33150616]), 'split3_test_score': array([-18.71607579, -18.58877469, -18.46438072, -1

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 185374529012.93475, tolerance: 60489037.085545205
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 212221144289.99973, tolerance: 86156056.45739678
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 187550524758.89847, tolerance: 79798507.65063699
  positive)
/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not 

{'mean_fit_time': array([1.10828443, 1.08238058, 1.07996173, 1.1777956 , 1.14317174]), 'std_fit_time': array([0.02996635, 0.01512468, 0.02088906, 0.10499971, 0.09109442]), 'mean_score_time': array([0.00440283, 0.00407877, 0.00430412, 0.00432529, 0.00408716]), 'std_score_time': array([0.00044789, 0.00025996, 0.00026746, 0.0003404 , 0.00018228]), 'param_alpha': masked_array(data=[0.1, 0.2, 0.3, 0.4, 0.5],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.2}, {'alpha': 0.3}, {'alpha': 0.4}, {'alpha': 0.5}], 'split0_test_score': array([0.27542837, 0.28281828, 0.28576312, 0.28761143, 0.28830464]), 'split1_test_score': array([-0.06383614, -0.071352  , -0.07470951, -0.07192148, -0.06998664]), 'split2_test_score': array([-0.16179865, -0.15862771, -0.15186788, -0.13136608, -0.11724562]), 'split3_test_score': array([-19.49129427, -19.65899689, -19.74151   , -19.67780931,
       -19.61660157]), 'split4_t

In [16]:
pd.DataFrame(results_grid['Until week: 0']['Lasso'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.324299,0.053483,0.004191,0.000824,0.1,{'alpha': 0.1},-2.545750,-0.080014,-37.885538,-2.129004,0.197399,-8.488581,14.738336,5
1,1.352226,0.041606,0.004453,0.000403,0.2,{'alpha': 0.2},-2.488585,-0.092630,-28.968431,-1.979790,0.200169,-6.665853,11.199672,4
2,1.338609,0.019975,0.004297,0.000120,0.3,{'alpha': 0.3},-2.468070,-0.104030,-21.328622,-1.891113,0.196326,-5.119102,8.168245,3
3,1.461599,0.023494,0.004938,0.000564,0.4,{'alpha': 0.4},-2.464836,-0.099678,-14.930238,-1.833980,0.191584,-3.827430,5.641834,2
4,1.429622,0.084922,0.004344,0.000231,0.5,{'alpha': 0.5},-2.451951,-0.096971,-9.788240,-1.773654,0.187884,-2.784586,3.639853,1


In [21]:
metrics_grid_lasso = pd.DataFrame(columns= ['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_alpha', 'params', 'split0_test_score', 'split1_test_score',
       'split2_test_score', 'split3_test_score', 'split4_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score'])
for key in results_grid:
    df = pd.DataFrame(results_grid[key]['Lasso'])
    metrics_grid_lasso= metrics_grid_lasso.append(df)

In [22]:
metrics_grid_lasso

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.324299,0.053483,0.004191,0.000824,0.1,{'alpha': 0.1},-2.545750,-0.080014,-37.885538,-2.129004,0.197399,-8.488581,14.738336,5
1,1.352226,0.041606,0.004453,0.000403,0.2,{'alpha': 0.2},-2.488585,-0.092630,-28.968431,-1.979790,0.200169,-6.665853,11.199672,4
2,1.338609,0.019975,0.004297,0.000120,0.3,{'alpha': 0.3},-2.468070,-0.104030,-21.328622,-1.891113,0.196326,-5.119102,8.168245,3
3,1.461599,0.023494,0.004938,0.000564,0.4,{'alpha': 0.4},-2.464836,-0.099678,-14.930238,-1.833980,0.191584,-3.827430,5.641834,2
4,1.429622,0.084922,0.004344,0.000231,0.5,{'alpha': 0.5},-2.451951,-0.096971,-9.788240,-1.773654,0.187884,-2.784586,3.639853,1
0,1.418115,0.042697,0.005767,0.001758,0.1,{'alpha': 0.1},-2.046427,0.035139,-17.776466,-3.266531,0.209030,-4.569051,6.730893,5
1,1.337317,0.017829,0.004314,0.000104,0.2,{'alpha': 0.2},-2.038451,0.026508,-11.264322,-3.223088,0.209716,-3.257927,4.204825,4
2,1.435397,0.077565,0.004504,0.000293,0.3,{'alpha': 0.3},-2.029100,0.021925,-6.327128,-3.200407,0.210077,-2.264926,2.399481,3
3,1.341168,0.037525,0.004631,0.000345,0.4,{'alpha': 0.4},-2.020605,0.023065,-2.958567,-3.202634,0.210809,-1.589587,1.449405,2
4,1.315364,0.019227,0.004495,0.000400,0.5,{'alpha': 0.5},-2.021277,0.024080,-1.165302,-3.191947,0.212464,-1.228396,1.275378,1
